<a href="https://colab.research.google.com/github/patriciaapenat/TFM/blob/main/Notebooks/Data_Frame_covid_daily_oms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/patriciaapenat/TFM.git

Cloning into 'TFM'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 187 (delta 45), reused 82 (delta 44), pack-reused 104
Receiving objects: 100% (187/187), 7.81 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (90/90), done.


### **Fuente de Datos - Ficheros WHO OMS**
**https://covid19.who.int/data**<br>
**https://ourworldindata.org/coronavirus#coronavirus-country-profiles**<br>


#### Descarga de datos

**1.- Casos diarios y muertes por fecha notificados a la OMS:** https://covid19.who.int/WHO-COVID-19-global-data.csv<br>
**2.- Últimos recuentos informados de casos y muertes:** https://covid19.who.int/WHO-COVID-19-global-table-data.csv<br>
**3.- Datos de vacunación:** https://covid19.who.int/who-data/vaccination-data.csv<br>
**4.- Metadatos de vacunación:**https://covid19.who.int/who-data/vaccination-metadata.csv<br>


**🧾Informacion del Dataset**<br>
*  Los usuarios deben tener en cuenta que, además de capturar nuevos casos y muertes notificadas en un día determinado, las actualizaciones se realizan retrospectivamente para corregir los recuentos de días anteriores según sea necesario en función de la información recibida posteriormente. Consulte "Datos agregados diarios de recuento de casos y muertes" más arriba para obtener más detalles sobre el cálculo de nuevos casos/muertes.

In [2]:
import pandas as pd
import numpy as np

#!pip install mysql-connector-python
#!pip install sqlalchemy
#!pip install PyMySql
#!pip install ipython-sql

In [ ]:
#pip install pycountry==20.7.3
#!pip install pycountry-convert==0.7.2

# Acceso a los datasets de la OMS:

Exloración e Información del DataFrame **df_covid_daily**

In [3]:
df_covid_daily = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [4]:
df_covid_daily.shape

(303360, 8)

In [5]:
df_covid_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303360 entries, 0 to 303359
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      303360 non-null  object
 1   Country_code       302080 non-null  object
 2   Country            303360 non-null  object
 3   WHO_region         303360 non-null  object
 4   New_cases          303360 non-null  int64 
 5   Cumulative_cases   303360 non-null  int64 
 6   New_deaths         303360 non-null  int64 
 7   Cumulative_deaths  303360 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 18.5+ MB


Transformamos **Date_reported** a formato datetime con formato **dd/mm/yy**

In [6]:
df_covid_daily['Date_reported']= pd.to_datetime(df_covid_daily['Date_reported']).dt.strftime('%d-%m-%Y')

Verificamos que se han modificado el tipo de datos a formato fecha **dd/mm/yy**

In [7]:
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,03-01-2020,AF,Afghanistan,EMRO,0,0,0,0
1,04-01-2020,AF,Afghanistan,EMRO,0,0,0,0
2,05-01-2020,AF,Afghanistan,EMRO,0,0,0,0
3,06-01-2020,AF,Afghanistan,EMRO,0,0,0,0
4,07-01-2020,AF,Afghanistan,EMRO,0,0,0,0


Observamos que no hay **ningún valor duplicado**

In [8]:
df_covid_daily.duplicated().values.any()

False

Observamos que hay **1280 valores nulos**

In [10]:
df_covid_daily.isnull().values.sum()

1280

Identificamos en que **Variable** y la **suma de valores nulos**

In [11]:
df_covid_daily.isnull().sum()

Date_reported           0
Country_code         1280
Country                 0
WHO_region              0
New_cases               0
Cumulative_cases        0
New_deaths              0
Cumulative_deaths       0
dtype: int64

Observamos que el Country Namibia tiene código Null debido a que su código ISO es **NM**

In [12]:
df_covid_daily[df_covid_daily['Country_code'].isnull()]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
183040,03-01-2020,NaN,Namibia,AFRO,0,0,0,0
183041,04-01-2020,NaN,Namibia,AFRO,0,0,0,0
183042,05-01-2020,NaN,Namibia,AFRO,0,0,0,0
183043,06-01-2020,NaN,Namibia,AFRO,0,0,0,0
183044,07-01-2020,NaN,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
184315,01-07-2023,NaN,Namibia,AFRO,0,171310,0,4091
184316,02-07-2023,NaN,Namibia,AFRO,0,171310,0,4091
184317,03-07-2023,NaN,Namibia,AFRO,0,171310,0,4091
184318,04-07-2023,NaN,Namibia,AFRO,0,171310,0,4091


Sustituimos los valores Null **NaM** de Namibia por el Código ISO2 **NM**

In [13]:
df_covid_daily=df_covid_daily.fillna('NM')
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,03-01-2020,AF,Afghanistan,EMRO,0,0,0,0
1,04-01-2020,AF,Afghanistan,EMRO,0,0,0,0
2,05-01-2020,AF,Afghanistan,EMRO,0,0,0,0
3,06-01-2020,AF,Afghanistan,EMRO,0,0,0,0
4,07-01-2020,AF,Afghanistan,EMRO,0,0,0,0


Verificamos que se ha cambiado correctamente el código ISO2 de Namibia a **NM**

In [14]:
df_covid_daily[df_covid_daily['Country']=="Namibia"]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
183040,03-01-2020,NM,Namibia,AFRO,0,0,0,0
183041,04-01-2020,NM,Namibia,AFRO,0,0,0,0
183042,05-01-2020,NM,Namibia,AFRO,0,0,0,0
183043,06-01-2020,NM,Namibia,AFRO,0,0,0,0
183044,07-01-2020,NM,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
184315,01-07-2023,NM,Namibia,AFRO,0,171310,0,4091
184316,02-07-2023,NM,Namibia,AFRO,0,171310,0,4091
184317,03-07-2023,NM,Namibia,AFRO,0,171310,0,4091
184318,04-07-2023,NM,Namibia,AFRO,0,171310,0,4091


Ya no tenemos vamores Nulos en ninguna variable del Dataframe

In [15]:
df_covid_daily.isnull().sum()

Date_reported        0
Country_code         0
Country              0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
dtype: int64

Cargo el fichero **paises_region_oms_v2.csv**

In [21]:
df_paises = pd.read_csv('/content/TFM/Ficheros_Depurados/paises_region_oms_v2.csv')
df_paises.head()

,PAIS_ISO3,PAIS_ISO2,PAIS_NOM,COD_CONTINENTE,CONTINENTE,OMS_REGION,DESC_OMS_REGION,PAIS_NOM_2
0,AGO,AO,Angola,AF,Africa,AFRO,Africa,Angola
1,BDI,BI,Burundi,AF,Africa,AFRO,Africa,Burundi
2,BEN,BJ,Benin,AF,Africa,AFRO,Africa,Benin
3,BFA,BF,Burkina Faso,AF,Africa,AFRO,Africa,Burkina Faso
4,BWA,BW,Botswana,AF,Africa,AFRO,Africa,Botswana


Realizo la unión entre ambos dataframes por la variable **ISO2**

In [23]:
df_covid_daily_new = pd.merge(left=df_covid_daily, right=df_paises,
                      how='left', left_on='Country_code', right_on='PAIS_ISO2')

In [25]:
df_covid_daily_new.isnull().sum()

Date_reported            0
Country_code             0
Country                  0
WHO_region               0
New_cases                0
Cumulative_cases         0
New_deaths               0
Cumulative_deaths        0
PAIS_ISO3             6400
PAIS_ISO2             6400
PAIS_NOM              6400
COD_CONTINENTE       57600
CONTINENTE            6400
OMS_REGION            6400
DESC_OMS_REGION       6400
PAIS_NOM_2            6400
dtype: int64

Identifico 6400 valores nulos pero corresponden a códigos ISO2 especiales y los sustituiré por el valor **Z9**

In [26]:
df_covid_daily_new[df_covid_daily_new['PAIS_ISO3'].isnull()]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,PAIS_NOM,COD_CONTINENTE,CONTINENTE,OMS_REGION,DESC_OMS_REGION,PAIS_NOM_2
32000,03-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32001,04-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32002,05-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32003,06-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32004,07-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249595,01-07-2023,XB,Sint Eustatius,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249596,02-07-2023,XB,Sint Eustatius,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249597,03-07-2023,XB,Sint Eustatius,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249598,04-07-2023,XB,Sint Eustatius,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Codigos especiales **iso2** que no aparecen en listado de paises_region.csv

https://es.wikipedia.org/wiki/ISO_3166-1_alfa-2

El código **XA** está siendo utilizado por Suiza, como código de país por las Islas Canarias, a pesar de que IC ya está reservado con dicho propósito.23<br>
El código **XI** está siendo utilizado por el Gobierno del Reino Unido, como el prefijo de código de país del número EORI de Irlanda del Norte.24<br>
El código **XK** está siendo utilizado por la Comisión Europea,25​ el FMI, la SWIFT,26​ el CLDR y otras organizaciones como código de país provisional para Kosovo.27<br>
El código **XN** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para el Instituto Nórdico de Patentes, una organización internacional a la que pertenecen Dinamarca, Islandia, Noruega y Suecia.28<br>
El código **XU** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para la Unión Internacional para la Protección de las Obtenciones Vegetales<br>
El código **XV** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para el Instituto Visegrad de Patentes<br>
El código **XX** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como un indicador para estados desconocidos, otras entidades u organizaciones<br>

Elimino columnas del dataframe

In [27]:
df_covid_daily_new.drop(['Country_code','Country','COD_CONTINENTE','CONTINENTE','OMS_REGION','DESC_OMS_REGION'],axis=1,inplace=True)

In [28]:
df_covid_daily_new[df_covid_daily_new['PAIS_ISO3'].isnull()]

,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,PAIS_NOM,PAIS_NOM_2
32000,03-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN,NaN
32001,04-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN,NaN
32002,05-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN,NaN
32003,06-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN,NaN
32004,07-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
249595,01-07-2023,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN
249596,02-07-2023,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN
249597,03-07-2023,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN
249598,04-07-2023,AMRO,0,1220,0,6,NaN,NaN,NaN,NaN


Sustituir valores nulos por **Z9****

In [29]:
df_covid_daily_new['PAIS_ISO3'].fillna('Z99', inplace=True)
df_covid_daily_new['PAIS_ISO2'].fillna('Z9', inplace=True)
df_covid_daily_new['PAIS_NOM'].fillna('Otros', inplace=True)
df_covid_daily_new['PAIS_NOM_2'].fillna('Otros', inplace=True)

Hacemos una última revisión. Observamos que hay 345 duplicados que son los registros que equivalen a **Z9**

In [30]:
df_covid_daily_new[df_covid_daily_new.duplicated()]

,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,PAIS_NOM,PAIS_NOM_2
226560,03-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros,Otros
226561,04-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros,Otros
226562,05-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros,Otros
226563,06-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros,Otros
226564,07-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros,Otros
...,...,...,...,...,...,...,...,...,...,...
248533,03-08-2020,AMRO,0,3,0,0,Z99,Z9,Otros,Otros
248572,11-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros,Otros
248573,12-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros,Otros
248574,13-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros,Otros


Observamos que ya **no existen valores nulos**

In [31]:
df_covid_daily_new.isnull().sum()

Date_reported        0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
PAIS_ISO3            0
PAIS_ISO2            0
PAIS_NOM             0
PAIS_NOM_2           0
dtype: int64

In [32]:
df_covid_daily_new.columns

Index(['Date_reported', 'WHO_region', 'New_cases', 'Cumulative_cases',
       'New_deaths', 'Cumulative_deaths', 'PAIS_ISO3', 'PAIS_ISO2', 'PAIS_NOM',
       'PAIS_NOM_2'],
      dtype='object')

In [33]:
df_covid_daily_new = df_covid_daily_new.reindex(columns =['PAIS_NOM','PAIS_ISO3','PAIS_ISO2','Date_reported','WHO_region','New_cases','Cumulative_cases','New_deaths','Cumulative_deaths', 'PAIS_NOM_2'])

In [34]:
df_covid_daily_new

,PAIS_NOM,PAIS_ISO3,PAIS_ISO2,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_NOM_2
0,Afghanistan,AFG,AF,03-01-2020,EMRO,0,0,0,0,Afghanistan
1,Afghanistan,AFG,AF,04-01-2020,EMRO,0,0,0,0,Afghanistan
2,Afghanistan,AFG,AF,05-01-2020,EMRO,0,0,0,0,Afghanistan
3,Afghanistan,AFG,AF,06-01-2020,EMRO,0,0,0,0,Afghanistan
4,Afghanistan,AFG,AF,07-01-2020,EMRO,0,0,0,0,Afghanistan
...,...,...,...,...,...,...,...,...,...,...
303355,Zimbabwe,ZWE,ZW,01-07-2023,AFRO,0,265524,0,5707,Zimbabwe
303356,Zimbabwe,ZWE,ZW,02-07-2023,AFRO,0,265524,0,5707,Zimbabwe
303357,Zimbabwe,ZWE,ZW,03-07-2023,AFRO,80,265604,2,5709,Zimbabwe
303358,Zimbabwe,ZWE,ZW,04-07-2023,AFRO,0,265604,0,5709,Zimbabwe


In [35]:
df_covid_daily_new.drop(['PAIS_NOM_2'], axis=1, inplace=True)

In [36]:
df_covid_daily_new.columns=['PAIS_NOM','PAIS_ISO3','PAIS_ISO2','FECHA_INFORMADA','OMS_REGION','CASOS_NUEVOS','CASOS_ACUM','MUERTES_NUEVAS','MUERTES_ACUM']

In [37]:
df_covid_daily_new

,PAIS_NOM,PAIS_ISO3,PAIS_ISO2,FECHA_INFORMADA,OMS_REGION,CASOS_NUEVOS,CASOS_ACUM,MUERTES_NUEVAS,MUERTES_ACUM
0,Afghanistan,AFG,AF,03-01-2020,EMRO,0,0,0,0
1,Afghanistan,AFG,AF,04-01-2020,EMRO,0,0,0,0
2,Afghanistan,AFG,AF,05-01-2020,EMRO,0,0,0,0
3,Afghanistan,AFG,AF,06-01-2020,EMRO,0,0,0,0
4,Afghanistan,AFG,AF,07-01-2020,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...,...
303355,Zimbabwe,ZWE,ZW,01-07-2023,AFRO,0,265524,0,5707
303356,Zimbabwe,ZWE,ZW,02-07-2023,AFRO,0,265524,0,5707
303357,Zimbabwe,ZWE,ZW,03-07-2023,AFRO,80,265604,2,5709
303358,Zimbabwe,ZWE,ZW,04-07-2023,AFRO,0,265604,0,5709


In [38]:
df_covid_daily_new.to_csv('/content/TFM/Ficheros_Depurados/df_covid_daily_new.csv',index=False)